In [1]:
%load_ext autoreload
%autoreload 15


In [2]:
# !pip3 install azure-cosmos

In [3]:
# !pip3 install comtypes

In [4]:
# !pip3 install pdfminer.six

In [5]:
import json
import re
from bs4 import BeautifulSoup
import os
import requests
from pprint import pprint
import uuid
from statistics import mode
from nltk.corpus import stopwords
from pdf2txt import convert_pdf
# !pip install tika
import tika
from definitions_v3 import *

/opt/app/RB/.local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (4.0.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [6]:
pointsRE_heading=re.compile('(?:\s*\([a-z]{1,3}\)|[A-Z]{1}\s+[a-zA-Z0-9_\s]{5})')
stop_words = list(set(stopwords.words('english')))
y_cordinate_Re=re.compile('top:(\d{1,5})px')
height_re=re.compile('height:(\d{1,5})px')

In [7]:
def check_underline(positions,y_cord_text):
    positions=[i for i in positions if y_cord_text<=i< y_cord_text+16]
    if positions:
        return True
    else:
        return False
    

In [8]:
def get_pages_no(soup):
    total_items = int(soup.find('span', id='NewReleases_total').text)
    items_per_page = int(soup.find('span', id='NewReleases_end').text)
    return round(total_items/items_per_page)

In [9]:
from pdfminer.high_level import extract_pages

# print(len(list(extract_pages('test/Chubb16-250-1019 Chubb EBM Business Pack Product Disclosure Statement (PDS) and Policy Wording.pdf'))))

In [10]:
def get_underlines(soup):
    positions=[]
    for span in soup.find_all('span'):
        y_cord_list=y_cordinate_Re.findall(str(span))
        if y_cord_list:
            y_cord=int(y_cord_list[0])
        else:
            continue
        style="position:absolute; border: black 1px solid" in str(span)
        height_px_li=height_re.findall(str(span))
        if height_px_li:
            height_px=height_px_li[0]
        else:
            continue
        height=int(height_px)<15
        if all([style,height]):
            positions.append(y_cord)
    return list(set(positions))

In [11]:
def ext_extract(text_list,language):
    ext_flag=False
    if language=='english':
        ext_lis=['extension']
    else:
        ext_lis=['extensión','extensiones']
   
    ext_count=0
    position_list=[]
    position=0
    for text in text_list:
        # if any two of 'exclusion ,condition or extension is present in the text, that text is omitted from the logic'
        bool_list=['condition' in text.lower(),'exclusion' in text.lower(),'extension' in text.lower()]
        if sum(bool_list)>=2:
            continue
        for i in ext_lis:
            for match in re.finditer(i, text.lower()):
                position_list.append(position+match.start())               
                ext_count+=1
        position+=len(text)
    if ext_count>=1:
        ext_flag=True
    return(ext_flag,ext_count,position_list)

In [12]:
def excl_extract(text_list,language):
    exclusion_flag=False
    if language=='english':
        excl_lis=['exclud','not cover','except','does not mean','not includ','exclusion']
    else:
        excl_lis=['excepto','excepción','no in cluido','exclusión','excluidos','excluirlo','excluyendo','exclusiones','exclusion','excluyen','excluyentes']
   
    exclusion_count=0
    position_list=[]
    position=0
    for text in text_list:
        # if any two of 'exclusion ,condition or extension is present in the text, that text is omitted from the logic'
        bool_list=['condition' in text.lower(),'exclusion' in text.lower(),'extension' in text.lower()]
        if sum(bool_list)>=2:
            continue
        for i in excl_lis:
            for match in re.finditer(i, text.lower()):
                position_list.append(position+match.start())               
                exclusion_count+=1
        position+=len(text)
    if exclusion_count>=1:
        exclusion_flag=True
    return(exclusion_flag,exclusion_count,position_list)

In [13]:
def cond_extract(text_list,language):
    cond_flag=False
    if language=='english':
        cond_lis=['condition']
    else:
        cond_lis=['condición','condiciones','condicionado']
   
    cond_count=0
    position_list=[]
    position=0
    for text in text_list:
        # if any two of 'exclusion ,condition or extension is present in the text, that text is omitted from the logic'
        bool_list=['condition' in text.lower(),'exclusion' in text.lower(),'extension' in text.lower()]
        if sum(bool_list)>=2:
            continue
        for i in cond_lis:
            for match in re.finditer(i, text.lower()):
                position_list.append(position+match.start())               
                cond_count+=1
        position+=len(text)
    if cond_count>=1:
        cond_flag=True
    return(cond_flag,cond_count,position_list)

In [14]:
def font_extraction(soup):
    fontsizes=[]
    for divs in soup.findAll('div'):
        for j in divs.find_all('span'):
            ext_size=re.findall(r'font-size:(.*)px">',str(j))
            if ext_size:
                fontsizes.append(int(ext_size[0]))
    return(fontsizes)

In [15]:
def stopword_check(word,text,language):
    if language=='spanish':
        stop_words = list(set(stopwords.words('spanish')))
    else:
        stop_words = list(set(stopwords.words('english')))
        transition="although  instead  whereas  despite  conversely  otherwise  however moreover  likewise  comparatively  correspondingly  similarly  furthermore  additionallyver  rather  nevertheless  nonetheless  regardless  notwithstanding consequently  therefore  thereupon  forthwith  accordingly  henceforth"
        transition_words=transition.split()
        transition_words
        stop_words.extend(transition_words)

    if exclusion_check(text,language):
        return True
    if word.lower() in stop_words:
        return False
    else:
        return True

In [16]:
def exclusion_check(text,language):
    if language=='spanish':
        exclusion_bag=['excepto','excepción','no in cluido','exclusión','excluidos','excluirlo','excluyendo',' excluyen','exclusiones','exclusiones','exclusion','excluyentes']
    else:
        exclusion_bag=['exclusion','excluded','not covered','will not cover','will not pay']
    for word in exclusion_bag:
        if word in text.lower():
            return True
    return False
    

In [17]:
def criteria_check(text,language):
    if language=='spanish':
        exclusion_bag=['condición','condiciones','condicionado']
    else:
        exclusion_bag=['condition','conditions']
    for word in exclusion_bag:
        if word in text.lower():
            return True
    return False
    

In [18]:
def ext_check(text,language):
    if language=='english':
        ext_bag=['extension']
    else:
        ext_bag=['extensión','extensiones']
    for word in ext_bag:
        if word in text.lower():
            return True
    return False
    

In [19]:
start=re.compile('.*\d{1,3}.*(?:\-|\.|–)')
def clean(text):
    text=text.replace('\n','').replace('\t',' ')
    text=start.sub('',text).strip()
    

    return(text)
def clean_pharses(phrases):
    phrases=[clean(i) for i in phrases if len(i)> 3]
    phrases=list(set(phrases))
    return phrases

In [20]:
def create_local_index(path,subdir,language,country):
    pointsRE_heading=re.compile('(?:\s*\([a-z]{1,3}\)|(?<![A-z])[A-Z]{1}\s+[a-zA-Z0-9_\s]{5})')
    pageNumRE=re.compile('Page\s*(\d{1,3})',re.IGNORECASE)
    neglect_def=['policy','insured','schedule']
    local_indexed={}
    previous_span=''
    bold_phrases_indexed={}
    for file in os.listdir(path):
        if file.endswith('.pdf'):
            print(file)
            global file_count
            file_count+=1
            
            print("COUNT",file_count)
            try:
                pg_no=len(list(extract_pages(os.path.join(path,file))))
            except:
                pg_no=0
            try:
                html=convert_pdf(os.path.join(path,file),'html')
            except:
                continue
            soup = BeautifulSoup(html, 'html5lib')
            underline_positions=get_underlines(soup)
            try:
                fontsizes=font_extraction(soup)
                file_font_size_mode=mode(fontsizes)
            except:
                file_font_size_mode=8
                
            text_para,text_plain_para='',''
            text_lis,text_plain_lis,bold_lis,page_list,=[],[],[],[]
            bold=False
            sub_page_def_list=[]
            try:
                definitions=def_extraction(os.path.join(path,file))
            except:
                continue
            definitions={key:value for key,value in definitions.items() if key.lower().strip() not in neglect_def}
            def_terms=list(definitions.keys())
#             print(len(def_terms),'LEN DEFINITIONS')
#             print(definitions)
            definition_text=''
            header_match_object=(0,'',False)
            cond_header_match_object=(0,'',False)
            ext_header_match_object=(0,'',False)
            second_category=False
            head_found=False
            def_flag=False
            cond_head_found=False
            ext_head_found=False
            single_page_head_found=False
            condition_text=''
            ext_text=''
            excl_text=''
            previous_pg_num=[]
            def_in_page=[]
            cond_single_page_head_found=False
            ext_single_page_head_found=False
            
            font_size=file_font_size_mode

            for divs in soup.findAll('div'):
                div_text_list=[span.text for span in divs.find_all('span') ]
                page_str=str(divs.find_all('a'))
                page_num_results=pageNumRE.findall(page_str)
                if page_num_results:
                    pagenum=page_num_results[0]
#                     print(pagenum)
                    if pagenum!=previous_pg_num:
                        sub_definitions_in_page=[]
                    previous_pg_num=pagenum
#                     definition_text=''
                    
                    #only for pdfs
                    if(int(pagenum) in [1,2]) and 'Wording' in file and pg_no >20:
#                     print('Continued on wording 2')
                        continue

                    if text_lis:
                        text_lis=text_lis[:-1]
                        text_para=''.join(text_lis)
                        text_plain_lis=text_plain_lis[:-1]
                        text_plain_para=''.join(text_plain_lis)
                        if(len(re.findall('\.',text_para))>(len(text_para)/3) or (len(re.findall('\d',text_para))>80 and int(pagenum)<4 and  pg_no>20 and 'content' in text_para.lower())) :
                            print('PASS')
                            print(text_plain_para)
                            text_para,text_plain_para=' ',' '
                            text_lis,text_plain_lis=[],[]
                            single_page_head_found=False
                            head_found=False
                            cond_head_found=False
                            ext_head_found=False
                            continue
                        
                        else:
                            bold_lis=clean_pharses(bold_lis)
                            pagenum=int(pagenum)-1
                            excl_flag,excl_count,excl_pos_lis=excl_extract(text_plain_lis,language)
                            cond_flag,cond_count,cond_pos_lis=cond_extract(text_plain_lis,language)
                            ext_flag,ext_count,ext_pos_lis=ext_extract(text_plain_lis,language)
                            if int(pagenum)==1:
                                if head_found or single_page_head_found or excl_count>0:
#                                     print(pagenum,True,'!!!')
                                    excl_flag=True
                                else:
                                    excl_flag=False
                            else:
                                if head_found or single_page_head_found:
#                                     print(pagenum,True,'!!!',excl_count)
                                    excl_flag=True
#                                     print(excl_flag,'EXCL')
#                                 else:
#                                     print(pagenum,excl_flag,'\n')
# #                             print(excl_text)
#                             print(condition_text)
                            if cond_head_found or cond_single_page_head_found or cond_count>0:
                                cond_flag=True
#                                 print('COND','TRUE')
                            else:
#                                 print('COND','FALSE')
                                cond_flag=False
    
                            if ext_head_found or ext_single_page_head_found or ext_count>0:
                                ext_flag=True
#                                 print('EXT','TRUE',pagenum)
                            else:
#                                 print('EXT','FALSE',pagenum)
                                ext_flag=False
#                             def_in_page=[{'name':k,'text':v[0] for k,v in definitions.items() if v[1]=pagenum}]
                            for term,defs in definitions.items():
#                                 print(defs[1])
                                if defs[1]==pagenum:  ###defs[1] is the pagenumber
                                    definition_text=definition_text+'          '+term+' '+defs[0]
                                    def_in_page.append({'name':term,'text':defs[0]})
#                                     def_in_page{'name':}
#                             print(definition_text)
                            if definition_text:
                                def_flag=True
                            
                            if def_flag:
                                def_search_flag=True
#                                 print('DEF','TRUE')
                            else:
#                                 print('DEF', 'FALSE')
                                def_search_flag=False   
                            
                            if pg_no>2:
                                endorsements=False
                            else:
                                endorsements=True
#                             print(endorsements,'ENDORSEMENTS!!!')
                            if text_para in local_indexed.keys():
                                text_para=text_para+' '
                            
                            local_indexed[text_para]=(file,pagenum,bold_lis,text_plain_para,excl_flag,excl_count,excl_pos_lis,subdir,sub_page_def_list,country,language,definition_text,def_search_flag,endorsements,excl_text,condition_text,cond_flag,cond_count,cond_pos_lis,def_in_page,ext_text,ext_flag,ext_count,ext_pos_lis)
#                             print(sub_page_def_list,'DEF LIST!!!!!1')
                            sub_page_def_list=[]
                            page_list.append(pagenum)
                            text_para,text_plain_para='',''
                            text_lis,text_plain_lis,bold_lis=[],[],[]
                            definition_text=''
                            cond_single_page_head_found=False
                            ext_single_page_head_found=False
                            condition_text=''
                            ext_text=''
                            excl_text=''
                            single_page_head_found=False
                            def_in_page=[]
                            def_flag=False
                for span in divs.find_all('span'):

                    bold=False
                    upper=False
                    bullet=False
                    def_flag=False
                    span_position=div_text_list.index(span.text)
                    
                    if "Bold" in str(span) or 'CIDFont+F3' in str(span):
                        bold_lis.append(span.text)
                        bold=True
                    if span.text.isupper():
                        upper=True
                    font_family_match=re.findall(r"font-family: b'(.*)';",str(span))
                    if font_family_match:
                        font_family=font_family_match[0]
                    else:
                        font_family=''
#                     print(font_family)
                    font_size_match=re.findall(r'font-size:(.*)px">',str(span))
                    if font_size_match:
                        font_size=int(font_size_match[0])
                    if pointsRE_heading.findall(span.text):
                        bullet=True
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
                    y_cord_list=y_cordinate_Re.findall(str(divs))
                    if y_cord_list:
                        y_cord=int(y_cord_list[0])
                    underlined_text=check_underline(underline_positions,y_cord)
                    if span.text.split('\n')[0]!='' and span.text.split('\n')[0]!=' ':
                        head_check_text=span.text.split('\n')[0]
                    elif len(span.text.split('\n'))>1:
                        head_check_text=span.text.split('\n')[1]
                    else:
                        head_check_text=''
##############################3  
                    if str(previous_span).endswith('<br/></span>'):
                        span_position=0
                    ##################INSERT FUNCTION####################z
                    #EXCLUSION 
                    if  span_position==0 and (bold or font_size>= file_font_size_mode +2 ) and (font_size> file_font_size_mode or upper) and 5<len(head_check_text.strip())<80:                    
                        if exclusion_check(head_check_text,language) and not head_found: 
                            head_found=True
                            header_match_object=(font_size,font_family,bold,upper,bullet)
                            First_category=True
                            second_category=False
                            single_page_head_found=True
#                             print('FOUND Heading for exclusion ...',span.text)
                        
                    
                        elif ((font_size,font_family,bold,upper,bullet)==header_match_object  or font_size>header_match_object[0] ) and head_found and not exclusion_check(head_check_text,language):
                            head_found=False
                            single_page_head_found=True
#                             print('FOUND Closure for exclusion ...',span.text)
#                     if 'CLAIMS CONDITIONS' in span.text:
#                         print ('^^^^^^^',(font_size,font_family,bold,upper,bullet),header_match_object, font_size>header_match_object[0] , head_found , not exclusion_check(head_check_text,language))
#                         print(pointsRE_heading.findall(span.text))
                    words_title=[word.istitle() for word in head_check_text.split() if stopword_check(word,head_check_text,language) and not word.isdigit() ]
                    if span_position==0 and all(words_title) and head_check_text.strip() not in definitions and  (len(words_title) >1 or underlined_text )and 5<len(head_check_text.strip())<80 :
#                         print('MMMMMMMMARARARA',span.text)

                        if exclusion_check(head_check_text,language) and  not head_found:
                            head_found=True
                            second_category=True
                            First_category=False
                            single_page_head_found=True
                            header_match_object=(font_size,font_family,bold,upper,bullet)
#                             print("FOUND heading type2 ....",span.text)
                        elif head_found and not exclusion_check(head_check_text,language) and second_category and ((font_size,font_family,bold,upper,bullet)==header_match_object or font_size > header_match_object[0] ) :
                            head_found=False
                            second_category=False
                            single_page_head_found=True
#                             print('FOUND closure type 2',span.text)
                    if head_found:
                        excl_text=excl_text+' '+span.text     
                     
                    #CONDITIONS
#                     if 'Condition' in span.text:
# #                         print('text:',str(span.text),'head check',head_check_text,'length of head check',len(head_check_text))
# #                         print(span_position==0,(bold or font_size>= file_font_size_mode +2 ), (font_size> file_font_size_mode or upper), 5<len(head_check_text.strip())<80)
# #                         print(font_size,file_font_size_mode,cond_head_found,criteria_check(head_check_text,language))
                    if  span_position==0 and (bold or font_size>= file_font_size_mode +2 ) and (font_size> file_font_size_mode or upper) and 5<len(head_check_text.strip())<80:                    
                        if criteria_check(head_check_text,language) and not cond_head_found: 
                            cond_head_found=True
                            cond_header_match_object=(font_size,font_family,bold,upper,bullet)
                            cond_First_category=True
                            cond_second_category=False
                            cond_single_page_head_found=True
#                             print('FOUND Heading for condition...',span.text)
                    
                        elif ((font_size,font_family,bold,upper,bullet)==cond_header_match_object or font_size>cond_header_match_object[0] ) and not criteria_check(head_check_text,language)  and cond_head_found:
                            cond_head_found=False
                            cond_single_page_head_found=True
#                             print('FOUND Closure for condition ...',span.text)
                
                    
                    cond_words_title=[word.istitle() for word in head_check_text.split() if stopword_check(word,head_check_text,language) and not word.isdigit() ]
                    if span_position==0 and all(cond_words_title) and head_check_text.strip() not in definitions and  (len(words_title) >1 or underlined_text )and 5<len(head_check_text.strip())<80 :
#                         print('MMMMMMMMARARARA',span.text)

                        if criteria_check(head_check_text,language) and  not cond_head_found:
                            cond_head_found=True
                            cond_second_category=True
                            cond_First_category=False
                            cond_single_page_head_found=True
                            cond_header_match_object=(font_size,font_family,bold,upper,bullet)
#                             print("FOUND heading type2 ....",span.text)
                        elif cond_head_found and not criteria_check(head_check_text,language) and cond_second_category and ((font_size,font_family,bold,upper,bullet)==cond_header_match_object or font_size > cond_header_match_object[0] ) :
                            cond_head_found=False
                            cond_second_category=False
                            cond_single_page_head_found=True
#                             print('FOUND closure type 2',span.text)

                    if cond_head_found==True:
                        condition_text=condition_text+' '+span.text
                            
                    
                    #####EXTENSIONS
                    #CONDITIONS
#                     if 'Extension' in span.text:
#                         print(span.text)
#                         print('text:',str(span.text),'head check',head_check_text,'length of head check',len(head_check_text))
#                         print(span_position==0,(bold or font_size>= file_font_size_mode +2 ), (font_size> file_font_size_mode or upper), 5<len(head_check_text.strip())<80)
#                         print(font_size,file_font_size_mode,cond_head_found,criteria_check(head_check_text,language))
                    if  span_position==0 and (bold or font_size>= file_font_size_mode +2 ) and (font_size> file_font_size_mode or upper) and 5<len(head_check_text.strip())<80:                    
                        if ext_check(head_check_text,language) and not ext_head_found: 
                            ext_head_found=True
                            ext_header_match_object=(font_size,font_family,bold,upper,bullet)
                            ext_First_category=True
                            ext_second_category=False
                            ext_single_page_head_found=True
#                             print('FOUND Heading for ext...',span.text)
                    
                        elif ((font_size,font_family,bold,upper,bullet)==ext_header_match_object or font_size>ext_header_match_object[0] ) and not ext_check(head_check_text,language)  and ext_head_found:
                            ext_head_found=False
                            ext_single_page_head_found=True
#                             print('FOUND Closure for ext ...',span.text)
                
                    
                    ext_words_title=[word.istitle() for word in head_check_text.split() if stopword_check(word,head_check_text,language) and not word.isdigit() ]
                    if span_position==0 and all(ext_words_title) and head_check_text.strip() not in definitions and  (len(words_title) >1 or underlined_text )and 5<len(head_check_text.strip())<80 :
#                         print('MMMMMMMMARARARA',span.text)

                        if ext_check(head_check_text,language) and  not ext_head_found:
                            ext_head_found=True
                            ext_second_category=True
                            ext_First_category=False
                            ext_single_page_head_found=True
                            ext_header_match_object=(font_size,font_family,bold,upper,bullet)
#                             print("FOUND heading type2 ext....",span.text)
                        elif ext_head_found and not ext_check(head_check_text,language) and ext_second_category and ((font_size,font_family,bold,upper,bullet)==ext_header_match_object or font_size > ext_header_match_object[0] ) :
                            ext_head_found=False
                            ext_second_category=False
                            ext_single_page_head_found=True
#                             print('FOUND closure type 2 ext',span.text)
# 
                    if ext_head_found==True:
                        ext_text=ext_text+' '+span.text
                            
    #                     else:
###############################             
                        
                    if font_size_match:
                        if font_size<=file_font_size_mode-1:
                            text_lis.append(' ')
                    text=span.text.replace('\n',' ')
                    text_plain=text.replace('\n',' ')
                    for term in def_terms:
                        if term.lower().strip() in text.lower() and definitions[term][0].strip() not in text and term not in sub_definitions_in_page:
#                             def_page.append(definitions[term])
#                             print(pagenum)
                            src_str  = re.compile(re.escape(term), re.IGNORECASE)
                            text=src_str.sub('###{}@@{}%%%'.format(term,definitions[term][0]),text)
#                             print(text,'\n',term,'TERM!!!!@#$%^&*&^%#$%^&')
                            sub_definitions_in_page.append(term)
                            sub_page_def_list.append({'name':term,'text':definitions[term][0]})
#                             print(definitions_in_page)
                            break
#                     print(definitions_in_page)

                    
                    text_lis.append(text)
                    text_plain_lis.append(text_plain)
                    previous_span=span
                    

            if text_lis:
                if(int(pagenum) in [1,2]) and 'Wording' in file and pg_no >20:
#                     print('Continued on wording 2')
                    continue

                text_lis=text_lis[:-1]
                text_para=''.join(text_lis)
                text_plain_lis=text_plain_lis[:-1]
                text_plain_para=''.join(text_plain_lis)
                if(len(re.findall('\.',text_para))>(len(text_para)/3) or (len(re.findall('\d',text_para))>80 and int(pagenum)<4 and pg_no>20 and 'content' in text_para.lower())) :
                    print('PASS')
                    print(text_plain_para)
                    text_para,text_plain_para=' ',' '
                    text_lis,text_plain_lis=[],[]
                    single_page_head_found=False
                    head_found=False
                    cond_head_found=False
                    ext_head_found=False
                    continue
                else:
                    bold_lis=clean_pharses(bold_lis)
                    if pagenum in page_list and pagenum!='1':
                        pagenum=int(pagenum)+1
                    excl_flag,excl_count,excl_pos_lis=excl_extract(text_plain_lis,language)
                    cond_flag,cond_count,cond_pos_lis=cond_extract(text_plain_lis,language)
                    ext_flag,ext_count,ext_pos_lis=ext_extract(text_plain_lis,language)
                    if int(pagenum)==1:
                        if head_found or single_page_head_found or excl_count>0:
#                             print(pagenum,True,'!!!',excl_count)
                            excl_flag=True
                        else:
                            excl_flag=False
                    else:
                        if head_found or single_page_head_found:
#                             print(pagenum,True,'!!!')
                            excl_flag=True
#                             print(excl_flag,'EXCL')
                        
                    #####################
                    if cond_head_found or cond_single_page_head_found or cond_count>0:
                        cond_flag=True
#                         print('COND','TRUE')
                    else:
                        cond_flag=False
                    
                    if ext_head_found or ext_single_page_head_found or ext_count>0:
                        ext_flag=True
#                         print('EXT','TRUE',pagenum)
                    else:
#                         print('EXT','FALSE',pagenum)
                        ext_flag=False
#                         print('COND','FALSE')
#                     page_def_list.append({'name':term,'text':definitions[term][0]}
#                     def_in_page=[{'name':k,'text':v[0]} for k,v in definitions.items() if v[1]==pagenum]  
#                     print(def_in_page,'!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                    for term,defs in definitions.items():
                        if defs[1]==pagenum:  ###defs[1] is the pagenumber
                            definition_text=definition_text+'          '+term+' '+defs[0]
                            def_in_page.append({'name':term,'text':defs[0]})        
#                     print(definition_text)
                    if definition_text:
                        def_flag=True

                    if def_flag:
                        def_search_flag=True
#                         print('DEF','TRUE')
                    else:
                        def_search_flag=False
#                         print('DEF','FALSE')
                    
                    if pg_no>2:
                        endorsements=False
                    else:
                        endorsements=True
#                     print(endorsements,'ENDORSEMENTS!!!')        
                    if text_para in local_indexed.keys():
                        text_para=text_para+' '
                    local_indexed[text_para]=(file,pagenum,bold_lis,text_plain_para,excl_flag,excl_count,excl_pos_lis,subdir,sub_page_def_list,country,language,definition_text,def_search_flag,endorsements,excl_text,condition_text,cond_flag,cond_count,cond_pos_lis,def_in_page,ext_text,ext_flag,ext_count,ext_pos_lis)
#                     print(sub_page_def_list,'DEF LIST!!!!!1')
                    sub_page_def_list=[]
                    text_para,text_plain_para='','' 
                    text_plain_lis,text_lis,bold_lis=[],[],[]
                    definition_text=''
                    single_page_head_found=False
                    cond_single_page_head_found=False
                    ext_single_page_head_found=False
                    condition_text=''
                    ext_text=''
                    excl_text=''
                    def_in_page=[]
                    def_flag=False
    return(local_indexed)

In [21]:
def doc_pdf_path(path):
    file_list=os.listdir(path)
    for file in file_list:
        raw_filename=file[:file.rfind(".")]
        pdf_filename=raw_filename+'.pdf'
#         print(raw_filename,pdf_filename,'FILE!!')
        if pdf_filename in file_list:
            print('continued!!!!!!!!!!!!!')
            continue
        if file.endswith('doc') or file.endswith('docx'):
            print('processing!!!')
            wordToPdf(os.path.join(path,file))

In [30]:
# rootDir='V://COG//AU- Property//'
# for dirname,subdirlist,filelist in os.walk(rootDir):
#     print(dirname)
#     for subdir in subdirlist:
#         path=os.path.join(dirname,subdir)
# #         doc_pdf_path(path)

In [22]:
# import os
file_count=0
rootDir='Documents/AU-Property/'
country='australia'
local_indexed={}
language='english'
if 'spanish' in rootDir:
    language='spanish'
for dirname,subdirlist,filelist in os.walk(rootDir):
    for subdir in subdirlist:
        print(subdir, 'SUBDIR!!!!!!!!!!!!!')
        path=os.path.join(dirname,subdir)    
        local_indexed_subdir=create_local_index(path,subdir,language,country)
        local_indexed.update(local_indexed_subdir)

Other Carrier Wording SUBDIR!!!!!!!!!!!!!
360 Commercial Policy Wording.pdf
COUNT 1
PASS
Contents IMPORTANT INFORMATION  3 – 5 DEFINITIONS  COVERAGE ................................................................. 3 POLICY SCHEDULE .................................................... 3   HOW TO MAKE A CLAIM ............................................. 3  360 COMMERCIAL ...................................................... 3  CGU ............................................................................ 3  DISCLOSURE .............................................................. 4  NON-DISCLOSURE ..................................................... 4  DISPUTE RESOLUTION .............................................. 4 PRIVACY ..................................................................... 4 SUBROGATION ........................................................... 5  UNDER INSURANCE ................................................... 5 CODE OF PRACTICE .........................

QBE Property Plus ISR MarkIV Version.pdf
COUNT 9
Care Providers ISR Mark IV (Modified) Policy Wording vXLICSE0419.pdf
COUNT 10
PASS
                                        Care Providers  Industrial Special Risks  Mark IV (Modified)  Policy Wording  vXLISE0419                                  Contents  Important Information  Industrial Special Risks Policy  Definitions  Damage  Section 1 – Material Loss or   Basis of Settlement  Memoranda to Section 1  Section 2 – Consequential Loss  Basis of Settlement  Memoranda to Section 2  Exclusions to all Sections  Property Exclusions  Perils Exclusions  Memoranda Applicable to all Sections  Conditions Applicable to all Sections  3  5  5  8  10  13  16  16  19  21  21  23  26  27                                Pen Underwriting Pty Ltd  ABN 89 113 929 516 AFSL 290518                                                              Contact Details  Current as at April 2018  Level 11, 99 York St  Sydney NSW 2000  GPO Box 4431  Sydney NSW 2001  Level 9,

4. Engineering.pdf
COUNT 16
1. Gross Revenue Specification.pdf
COUNT 17
SUU-CLL-PolicyOverview-2019-03-06.pdf
COUNT 18
6. Approved Valuer Valuation.pdf
COUNT 19
9. Infectious or Contagious Diseases (Pen).pdf
COUNT 20
Equipment Breakdown Proposal Form.pdf
COUNT 21
2. Gross Rentals Specification.pdf
COUNT 22
VERO - MBD - V0002 23_05_19 A.pdf
COUNT 23
SUU_strata-c_0219.pdf
COUNT 24
7. Subrogation waiver - Amended 30.10.pdf
COUNT 25
Care Providers ISR Mark IV (Modified) Policy Wording vXLICSE0420.pdf
COUNT 26
PASS
                                        Care Providers  Industrial Special Risks  Mark IV (Modified)  Policy Wording  vXLICSE0420                                  Contents  Important Information  Industrial Special Risks Policy  Definitions  Damage  Section 1 – Material Loss or   Basis of Settlement  Memoranda to Section 1  Section 2 – Consequential Loss  Basis of Settlement  Memoranda to Section 2  Exclusions to all Sections  Property Exclusions  Perils Exclusions  Memoranda App

AIMS EXISTPC4 CONTRACT WORKS - EXISTING STRUCTURE (B).pdf
COUNT 82
JLT.  Government Incentives [Section 2-Consequential Loss, Basis Of Settlement].pdf
COUNT 83
SFAST13 HANDYVS4 Demolition and Removal of Owned Property (new).pdf
COUNT 84
JLT. Salvage Sale Definition [Gross Rentals Specification].pdf
COUNT 85
AIMS BICOVXB4 TERMINATION OF COVER UNDER SECTION 2.pdf
COUNT 86
SFAST13 BETTAVB4 Output Replacement (new).pdf
COUNT 87
WPLACE REPELPC4 Infectious or Contagious Diseases; Vermin, Pests or Defective Sanitary Arrangements; Food or D (new).pdf
COUNT 88
JLT. Reinstatement of Damage by the Insured [Basis Of Settlement].pdf
COUNT 89
JLT. Money Loss Discovery Date [Property Exclusions].pdf
COUNT 90
SFAST13 ENQUIXB4 Liability to Make Enquiries (new).pdf
COUNT 91
L.REAC Public Relations Expenses [Section 1 Material Loss or Damage, The Indemnity].pdf
COUNT 92
AJG.63 Sprinkler Installations [Conditions Applicable To All Sections].pdf
COUNT 93
L.REAC ISR Policy - Excluding Crime [Conditions Appl

AIMS BASIIPC4 BASIS OF SETTLEMENT (i).pdf
COUNT 202
SFAST13 RDAIRVS4 Theft in Open  Resultant Damage (new).pdf
COUNT 203
SFAST08 LATERXB4 LOSS DISCOVERED LATE (new).pdf
COUNT 204
WPLACE ENQUIXB4 Liability to Make Enquiries (new).pdf
COUNT 205
AJG.65 Hazardous Goods [Conditions Applicable To All Sections].pdf
COUNT 206
AIMS SUBSDVS4 SUBSIDENCE CAUSED BY SEISMOLOGICAL DISTURBANCE.pdf
COUNT 207
L.REAC Government Incentives [Memoranda to Section 2].pdf
COUNT 208
MMAISR PROEXM PROPERTY EXCLUSION 16 MACHINERY BREAKDOWN (new).pdf
COUNT 209
MMAISR ITEMNO3 ITEM NO. 3 - DUAL BASIS PAY-ROLL - WAGES ONLY (new).pdf
COUNT 210
SFAST08 ENQUIXB4 LIABILITY TO MAKE ENQUIRIES (new).pdf
COUNT 211
L.REAC Acquired Companies. Acquired Property and Declared Values [Memoranda to Section 1].pdf
COUNT 212
WPLACE SUBSDVS4 Subsidence Caused by Seismological Disturbance (new).pdf
COUNT 213
JLT. Damage by Burglars [Additional Memoranda to Section 1].pdf
COUNT 214
SFAST13 FOUNDXS4 Undamaged Foundations (B) (new).pdf
C

...

RMSIRXS4 SELF-INSURANCE EXEMPTION.pdf
COUNT 1359
CITINXS4 MONEY INSURED BY CARRIER (new).pdf
COUNT 1360
USCSLXB4 UNSPECIFIED SUPPLIERS' AND OR CUSTOMERS' PREMISES (SINGLE LIMIT) (new).pdf
COUNT 1361
PRPAIRP4 PROPERTY IN THE OPEN AIR (STORM) (A) (new).pdf
COUNT 1362
MKV STORMY05 WIND, RAIN OR HAIL DAMAGE TO UNSPECIFIED PROPERTY IN THE  OPEN (new).pdf
COUNT 1363
TREESXS4 PLANTATIONS (new).pdf
COUNT 1364
BASIPCF4 BASIS OF SETTLEMENT (F) (new).pdf
COUNT 1365
MKV SUBEMP14 WAIVER OF SUBROGATION AGAINST EMPLOYEES (new).pdf
COUNT 1366
MINIMSX4 LOSS MINIMISATION (A).pdf
COUNT 1367
TNANTXB4 SUBROGATION WAIVER (TENANTS AND LESSEES).pdf
COUNT 1368
ALTERVS4 ALTERATION (A).pdf
COUNT 1369
SLABBXB4 FOUNDATIONS EXCLUSION MEMORANDUM (B) (new).pdf
COUNT 1370
BOWLSXS4 BOWLING GREENS.pdf
COUNT 1371
XMTWKXB4 CONTRACT WORKS - EXEMPTION (new).pdf
COUNT 1372
OCPBIPC4 OTHER CONTRIBUTING PROPERTIES (new).pdf
COUNT 1373
BASIPCF4 BASIS OF SETTLEMENT (F).pdf
COUNT 1374
LEASEXS4 LEASED EQUIPMENT - RESIDUAL VALUE (B)

SALESXB4 CUSTOMERSí GOODS (new).pdf
COUNT 1493
CREEPXB4 CREEPING, HEAVING AND VIBRATION (new).pdf
COUNT 1494
UMAKEXB4 REINSTATEMENT OF DAMAGE BY THE INSURED (new).pdf
COUNT 1495
MKV TERROR06 DAMAGE BY TERRORISTS (new).pdf
COUNT 1496
SPOILXS4 SPOILAGE OF GOODS IN A COLD STORAGE (new).pdf
COUNT 1497
MKV FLEETS09 CONTRACT CARRIERS' VEHICLES (new).pdf
COUNT 1498
MKV REPROS14 REPRESENTATION OVERSEAS (new).pdf
COUNT 1499
UTILPPC4  REMOTE PROPERTY OF PUBLIC UTILITIES (new).pdf
COUNT 1500
SWAREXS4 COMPUTER SOFTWARE.pdf
COUNT 1501
MKV YARDGO06 THEFT IN OPEN (EXCEPT MONEY) - EXCLUSION MODIFIED (new).pdf
COUNT 1502
RDAIRVS4 THEFT IN OPEN - RESULTANT DAMAGE (new).pdf
COUNT 1503
JEWLFPC4 JEWELLERY AND FURS OF EMPLOYEES AND OTHERS (B) (new).pdf
COUNT 1504
SITESXS4 CONTRACT SITES (new).pdf
COUNT 1505
BASIPCE4 BASIS OF SETTLEMENT (E).pdf
COUNT 1506
UNDELXB4 GOODS SOLD BUT NOT DELIVERED (A) (new).pdf
COUNT 1507
PROPEXH4 PROPERTY EXCLUSIONS (new).pdf
COUNT 1508
LEASEXB4 LEASED EQUIPMENT (RESIDUAL VALUE)

HEMINXB4 ISOLATION BY LANDSLIDE OR FLOOD (new).pdf
COUNT 1633
MKV WKSVAL05 B CONTRACT WORKS VALUE (new).pdf
COUNT 1634
NODECPC4 DECLARED VALUES (B) (new).pdf
COUNT 1635
GWILLXS4 GOODWILL.pdf
COUNT 1636
DRAINXS4 FLOOD - AMENDED DEFINITION (new).pdf
COUNT 1637
HOLDUPC4 HOLD-UPTHEFT.pdf
COUNT 1638
MKV HOTWET06 CHANGES IN A TEMPERATURE AND OR HUMIDITY CONTROLLED ENVIRONMENT (new).pdf
COUNT 1639
AFIREXB4 SPONTANEOUS COMBUSTION, FERMENTATION OR HEATING (new).pdf
COUNT 1640
TVRADXS4 TELEVISION OR RADIO - CANCELLED PROGRAMME EXPENSES.pdf
COUNT 1641
FBRIGXS4 FIRE BRIGADE ATTENDANCE FEES.pdf
COUNT 1642
MKV CASHGO05 MONEY LOSS DISCOVERED LATE (new).pdf
COUNT 1643
COWSAXB4 LIVESTOCK FIRE (A).pdf
COUNT 1644
DEMOLXS4 DEMOLITION OF DAMAGED PROPERTY.pdf
COUNT 1645
LANDPCD4 LANDSCAPING (D) (new).pdf
COUNT 1646
OPALTXB4 TURNOVEROUTPUT ALTERNATIVE AMENDED (new).pdf
COUNT 1647
MKV DEBREM03 CONTRACTUAL LIABILITY FOR DEBRIS REMOVAL (new).pdf
COUNT 1648
SPRAYPC4 SPRINKLER INSTALLATIONS (C) (new).pdf
COUNT 16

NOTGOXB4 NON-OPERATION (new).pdf
COUNT 1769
MKV CIRCEX06 6. CIRCUMSTANCES WHERE COVER IS EXCLUDED HEADING (new).pdf
COUNT 1770
ACQCAXB4 ACQUIRED COMPANIES (A).pdf
COUNT 1771
ELECPCD4 ELECTRONIC DATA EXCLUSION (D) (new).pdf
COUNT 1772
MGNETXS4 LOSS OF ATTRACTION - DRAWCARD PREMISES.pdf
COUNT 1773
FIDELLP4 DISHONEST ACTS BY EMPLOYEES.pdf
COUNT 1774
SLNOLLA4 SEVERAL LIABILITY NOTICE (A) (new).pdf
COUNT 1775
JEWLFUP4 JEWELLERY AND FURS OF EMPLOYEES AND OTHERS (A) (new).pdf
COUNT 1776
THRETXS4 THREATS OF DAMAGE, INJURY OR DEATH.pdf
COUNT 1777
CLASSPC4 CLASSIFICATION OF PROPERTY INSURED.pdf
COUNT 1778
CURRAXB4 CURRENCY (A) (new).pdf
COUNT 1779
SITESXS4 CONTRACT SITES.pdf
COUNT 1780
TDVRTXS4 TRANSFERABLE SITE AREA OF A HERITAGELISTED BUILDING.pdf
COUNT 1781
FDNOKVB4 UNDAMAGED FOUNDATIONS (A).pdf
COUNT 1782
DICONXB4 DIFFERENCE IN CONDITIONS (new).pdf
COUNT 1783
MKV CITINS05 MONEY INSURED BY CARRIER (new).pdf
COUNT 1784
MKV SURGES06 STORM SURGE (new).pdf
COUNT 1785
AGREEXB4 CONTRACTUAL AGREEMEN

MKV CONTWO11 11. CONDITIONS APPLYING TO SECTION 2 (new).pdf
COUNT 1905
ORDERXS4 PRECEDENCE OF DEDUCTIBLES.pdf
COUNT 1906
PRSETXB4 PAIRS AND SETS (A).pdf
COUNT 1907
CPREPXB4 CLAIMS COSTS EXTENDED (new).pdf
COUNT 1908
LANDBXS4 LANDSCAPING (B) (new).pdf
COUNT 1909
AUTHRXB4 AUTHORSHIP (new).pdf
COUNT 1910
CANCLXB4 CANCELLATION.pdf
COUNT 1911
FLOODXB4 FLOOD.pdf
COUNT 1912
NOAVEXB4 AMENDED PROVISO.pdf
COUNT 1913
POLNBXS4 POLLUTION, CONTAMINATION AND DEBRIS (B).pdf
COUNT 1914
CARSBXS4 REGISTERED VEHICLES (B) (new).pdf
COUNT 1915
GSTEPCB4 GOODS AND SERVICES TAX  ENDORSEMENT FOR GST REGISTERED POLICYHOLDERS ONLY (B).pdf
COUNT 1916
MKV ARTSTK05 WORKS OF ART, ANTIQUES AND CURIOS, AS STOCK (new).pdf
COUNT 1917
FLEETXB4 MOTOR VEHICLES (new).pdf
COUNT 1918
OTHERXB4 SUBROGATION WAIVER (OTHER INTERESTED PARTIES) (new).pdf
COUNT 1919
STORMXB4 WEATHER DAMAGE TO CERTAIN PROPERTY.pdf
COUNT 1920
MFACTVS4 MISREPRESENTATION AND NON-DISCLOSURE.pdf
COUNT 1921
MKV XTNTCV02 2. EXTENT OF COVER (new).pdf
COUNT 192

In [23]:
len(local_indexed)

3943

In [24]:
from azure.cosmos import exceptions, CosmosClient, PartitionKey

cosmo_endpoint="https://nf-poc-cdb-sql.documents.azure.com"
cosmo_key="iEcEfrxYe0Fm9QtoxDrOpLvGsfzjowwybULlWT9Uz4XxV4RmOIAnRuLdgRFUu1LPU5Vwk3UGivRrPrxnk7083w=="
client = CosmosClient(cosmo_endpoint, cosmo_key)
database=client.get_database_client('policy-analysis')
container=database.get_container_client('property-wordings')

In [25]:
for key,value in local_indexed.items():
    doc={}
    doc['text']=key
    doc['page']=int(value[1])
    if value[4]==True or value[0].find('Exclusion')!=-1:
        doc['IsExclusion']=True
    else:
        doc['IsExclusion']=False
    doc['doc_name']=value[0].replace('.pdf','')
    doc['id']=doc['doc_name']+' '+str(doc['page'])
    doc['bold_phrases']=value[2]
    doc['plain_text']=value[3]
    doc['excl_count']=value[5]
    doc['excl_pos']=value[6]
    doc['folder']=value[7]
    doc['definitions']=[{'name':key,'text':value} for key,value in re.findall('###(.*?)@@(.*?)%%%',doc['text'])]
    doc['country']=value[9]
    doc['language']=value[10]
    doc['definition_text']=value[11]
#     print(doc['definition_text'],'\n\n')
    if doc['definition_text']:
        doc['definition_flag']=True
    else:
        doc['definition_flag']=False
    if value[14]==True or 'endorsement' in value[0].lower(): 
        doc['endorsements']=True
    else:
        doc['endorsements']=value[13]
    doc['excl_text']=value[14]
    doc['cond_text']=value[15]
    doc['cond_flag']=value[16]
    doc['cond_count']=value[17]
    doc['cond_pos']=value[18]
    doc['definitions_in_page']=value[19]
    doc['ext_text']=value[20]
    doc['ext_flag']=value[21]
    doc['ext_count']=value[22]
    doc['ext_pos']=value[23]
    
    try:       
        container.create_item(body=doc)
    except Exception as err:
        print(err)
        pass

(Conflict) Entity with the specified id already exists in the system., 
RequestStartTime: 2020-12-16T16:31:46.8146266Z, RequestEndTime: 2020-12-16T16:31:46.8245814Z,  Number of regions attempted:1
ResponseTime: 2020-12-16T16:31:46.8245814Z, StoreResult: StorePhysicalAddress: rntbd://cdb-ms-prod-eastus1-fd58.documents.azure.com:14073/apps/f731575d-fb29-4640-9ffc-a6de234ddc91/services/78ee78c6-b247-456d-ab37-8e45873f3782/partitions/0e690849-4cb2-4a3c-b969-080feeb81a53/replicas/132525678004960632p/, LSN: 1889, GlobalCommittedLsn: 1889, PartitionKeyRangeId: 0, IsValid: True, StatusCode: 409, SubStatusCode: 0, RequestCharge: 2.38, ItemLSN: -1, SessionToken: -1#1889, UsingLocalLSN: False, TransportException: null, ResourceType: Document, OperationType: Create
, Microsoft.Azure.Documents.Common/2.11.0


In [26]:
print('added to cosmos')

added to cosmos
